# Create the dataset for the model and calculate statistics for each player

In [ ]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import json
import os
from collections import defaultdict, deque

pd.set_option('display.max_columns', None)

In [4]:
clean_data = pd.read_csv("../data/cleanedDataset.csv")
clean_data

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,p1_id,p1_seed,p1_entry,p1_name,p1_hand,p1_ht,p1_ioc,p1_age,p2_id,p2_seed,p2_entry,p2_name,p2_hand,p2_ht,p2_ioc,p2_age,score,best_of,round,minutes,p1_ace,p1_df,p1_svpt,p1_1stIn,p1_1stWon,p1_2ndWon,p1_SvGms,p1_bpSaved,p1_bpFaced,p2_ace,p2_df,p2_svpt,p2_1stIn,p2_1stWon,p2_2ndWon,p2_SvGms,p2_bpSaved,p2_bpFaced,p1_rank,p1_rank_points,p2_rank,p2_rank_points,RESULT
0,1991-301,Auckland,Hard,32,A,19910107,1,101746,NaN,NaN,Renzo Furlan,R,175.0,ITA,20.6,101142,1.0,NaN,Emilio Sanchez,R,180.0,ESP,25.6,6-4 6-1,3,R32,63.0,3.0,0.0,46.0,30.0,17.0,7.0,8.0,2.0,6.0,1.0,0.0,53.0,37.0,30.0,7.0,9.0,5.0,6.0,78.0,459.0,9.0,1487.0,0
1,1991-301,Auckland,Hard,32,A,19910107,2,100587,NaN,WC,Steve Guy,R,188.0,NZL,31.8,101613,NaN,Q,Malivai Washington,R,180.0,USA,21.5,6-3 6-2,3,R32,72.0,4.0,7.0,56.0,30.0,22.0,6.0,8.0,7.0,11.0,5.0,1.0,56.0,25.0,17.0,20.0,9.0,1.0,2.0,220.0,114.0,94.0,371.0,0
2,1991-301,Auckland,Hard,32,A,19910107,3,101601,NaN,WC,Brett Steven,R,185.0,NZL,21.6,101179,NaN,NaN,Jean Philippe Fleurian,R,185.0,FRA,25.3,2-6 6-1 6-2,3,R32,101.0,1.0,3.0,68.0,43.0,24.0,14.0,11.0,4.0,8.0,2.0,4.0,80.0,55.0,35.0,16.0,12.0,2.0,4.0,212.0,116.0,77.0,468.0,0
3,1991-301,Auckland,Hard,32,A,19910107,4,101332,8.0,NaN,Gilad Bloom,L,173.0,ISR,23.8,101117,NaN,NaN,Eric Jelen,R,180.0,GER,25.8,6-3 1-6 6-4,3,R32,108.0,3.0,2.0,96.0,61.0,38.0,15.0,13.0,8.0,12.0,0.0,1.0,82.0,55.0,35.0,14.0,13.0,6.0,10.0,72.0,483.0,65.0,502.0,0
4,1991-301,Auckland,Hard,32,A,19910107,5,101901,NaN,Q,Chuck Adams,R,185.0,USA,19.7,101735,3.0,NaN,Richard Fromberg,R,196.0,AUS,20.6,6-3 6-4,3,R32,65.0,4.0,4.0,65.0,46.0,34.0,12.0,10.0,2.0,2.0,1.0,3.0,49.0,25.0,21.0,12.0,9.0,4.0,6.0,190.0,142.0,28.0,876.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95370,2024-M-DC-2024-WG2-PO-TOG-INA-01,Davis Cup WG2 PO: TOG vs INA,Hard,4,D,20240203,4,133933,NaN,NaN,Thomas Yaka Kofi Setodji,R,183.0,TOG,28.2,207134,NaN,NaN,Fitriadi M Rifqi,R,175.0,INA,25.0,6-2 6-3,3,RR,87.0,2.0,1.0,50.0,35.0,19.0,5.0,8.0,5.0,9.0,1.0,2.0,52.0,32.0,25.0,11.0,9.0,2.0,3.0,819.0,24.0,569.0,64.0,0
95371,2024-M-DC-2024-WG2-PO-TUN-CRC-01,Davis Cup WG2 PO: TUN vs CRC,Hard,4,D,20240202,1,132374,NaN,NaN,Jesse Flores,R,188.0,CRC,28.8,121411,NaN,NaN,Moez Echargui,R,178.0,TUN,31.0,6-2 6-3,3,RR,65.0,3.0,2.0,49.0,30.0,17.0,9.0,8.0,2.0,5.0,4.0,2.0,56.0,35.0,30.0,11.0,9.0,4.0,4.0,900.0,18.0,279.0,205.0,0
95372,2024-M-DC-2024-WG2-PO-URU-MDA-01,Davis Cup WG2 PO: URU vs MDA,Clay,4,D,20240203,1,208364,NaN,NaN,Franco Roncadelli,L,185.0,URU,23.9,209943,NaN,NaN,Ilya Snitari,R,188.0,MDA,21.8,4-6 6-1 6-1,3,RR,137.0,1.0,2.0,74.0,50.0,31.0,13.0,12.0,5.0,9.0,0.0,5.0,92.0,52.0,24.0,18.0,12.0,8.0,16.0,616.0,55.0,740.0,34.0,1
95373,2024-M-DC-2024-WG2-PO-URU-MDA-01,Davis Cup WG2 PO: URU vs MDA,Clay,4,D,20240203,4,105430,NaN,NaN,Radu Albot,R,175.0,MDA,34.2,208364,NaN,NaN,Franco Roncadelli,L,185.0,URU,23.9,6-3 6-1,3,RR,95.0,2.0,1.0,50.0,30.0,19.0,14.0,8.0,3.0,4.0,1.0,1.0,82.0,66.0,33.0,6.0,8.0,6.0,11.0,136.0,489.0,616.0,55.0,1


### Create the dataset, calculate statistics for each player

In [ ]:
from utils.updateStats import getStats, updateStats, createStats

final_dataset = []
prev_stats = createStats()

# Iterate through each row in clean_data
for index, row in tqdm(clean_data.iterrows(), total=len(clean_data)):
    player1 = {
        "ID": row["p1_id"],
        "ATP_POINTS": row["p1_rank_points"],
        "ATP_RANK": row["p1_rank"],
        "AGE": row["p1_age"],
        "HEIGHT": row["p1_ht"],
    }

    player2 = {
        "ID": row["p2_id"],
        "ATP_POINTS": row["p2_rank_points"],
        "ATP_RANK": row["p2_rank"],
        "AGE": row["p2_age"],
        "HEIGHT": row["p2_ht"],
    }

    match = {
        "BEST_OF": row["best_of"],
        "DRAW_SIZE": row["draw_size"],
        "SURFACE": row["surface"],
    }
    
    # Call getStatsPlayers function
    output = getStats(player1, player2, match, prev_stats)

    # Append sorted stats to final dataset
    match_data = dict(sorted(output.items()))
    match_data["RESULT"] = row.RESULT
    final_dataset.append(match_data)

    # update stats
    prev_stats = updateStats(row, prev_stats)


# Convert final dataset to DataFrame
final_dataset = pd.DataFrame(final_dataset)

In [ ]:
player_id_to_plot = 206173

# Plot General ELO History
if player_id_to_plot in prev_stats["elo_history_players"]:
    general_elo_history = list(prev_stats["elo_history_players"][player_id_to_plot])
    
    if general_elo_history: # Check if history exists
        plt.figure(figsize=(12, 6))
        plt.plot(range(len(general_elo_history)), general_elo_history, label=f"Player {player_id_to_plot} General ELO")
        plt.xlabel("Match Sequence Index for Player")
        plt.ylabel("ELO Rating")
        plt.title(f"General ELO History for Player {player_id_to_plot}")
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print(f"No general ELO history found for Player {player_id_to_plot}")

else:
    print(f"Player {player_id_to_plot} not found in general ELO history records.")


# Plot surface specific ELO
plt.figure(figsize=(15, 8))
plotted_any_surface = False


surfaces_to_plot = clean_data['surface'].unique().tolist() 

In [ ]:
def convert_to_json_serializable(data):
    """
    Recursively converts defaultdicts and deques to standard dicts and lists.
    Converts tuple keys in dictionaries to string representations.
    """
    if isinstance(data, defaultdict):
        # Convert defaultdict to dict and recursively process its values
        # Also handle tuple keys by converting them to strings
        return {str(k) if isinstance(k, tuple) else k: convert_to_json_serializable(v) for k, v in data.items()}
    elif isinstance(data, dict):
        # Convert regular dict and recursively process its values
        # Also handle tuple keys by converting them to strings
        return {str(k) if isinstance(k, tuple) else k: convert_to_json_serializable(v) for k, v in data.items()}
    elif isinstance(data, deque):
        # Convert deque to list and recursively process its elements
        return [convert_to_json_serializable(elem) for elem in data]
    elif isinstance(data, list):
        # Recursively process elements of a list
        return [convert_to_json_serializable(elem) for elem in data]
    elif hasattr(data, 'item'): # Basic check for numpy numbers
        return data.item()
    return data


In [ ]:
print("Converting prev_stats to JSON serializable format...")

json_compatible_prev_stats = convert_to_json_serializable(prev_stats)
print("Conversion complete.")


prev_stats_json_output_path = "../data/prev_stats.json" # Example path

# Ensure the target directory exists
os.makedirs(os.path.dirname(prev_stats_json_output_path), exist_ok=True)

print(f"Attempting to save prev_stats to {prev_stats_json_output_path}...")
try:
    with open(prev_stats_json_output_path, "w") as f:
        json.dump(json_compatible_prev_stats, f, indent=2)
    print(f"Successfully saved prev_stats to {prev_stats_json_output_path}")
except Exception as e:
    print(f"Error saving prev_stats as JSON: {e}")
    import traceback
    traceback.print_exc()

In [12]:
# final_dataset.iloc[80000:90000]
final_dataset[4000]

{'AGE_DIFF': -1.0,
 'ATP_POINTS_DIFF': 501.0,
 'ATP_RANK_DIFF': -266.0,
 'BEST_OF': 3,
 'DRAW_SIZE': 32,
 'ELO_DIFF': 38.02783102669582,
 'ELO_GRAD_LAST_100_DIFF': 0,
 'ELO_GRAD_LAST_10_DIFF': 0,
 'ELO_GRAD_LAST_200_DIFF': 0,
 'ELO_GRAD_LAST_25_DIFF': 0,
 'ELO_GRAD_LAST_3_DIFF': 0,
 'ELO_GRAD_LAST_50_DIFF': 0,
 'ELO_GRAD_LAST_5_DIFF': 0,
 'ELO_SURFACE_DIFF': -5.672591118253649,
 'H2H_DIFF': 0,
 'H2H_SURFACE_DIFF': 0,
 'HEIGHT_DIFF': -3.0,
 'N_GAMES_DIFF': 28,
 'P_1STIN_LAST_100_DIFF': 66.12963264114433,
 'P_1STIN_LAST_10_DIFF': 66.93343032501149,
 'P_1STIN_LAST_200_DIFF': 66.12963264114433,
 'P_1STIN_LAST_25_DIFF': 65.45937244693405,
 'P_1STIN_LAST_3_DIFF': 75.36756336756336,
 'P_1STIN_LAST_50_DIFF': 66.12963264114433,
 'P_1STIN_LAST_5_DIFF': 70.6403175060041,
 'P_1STWON_LAST_100_DIFF': 61.94830124360268,
 'P_1STWON_LAST_10_DIFF': 61.52651519708843,
 'P_1STWON_LAST_200_DIFF': 61.94830124360268,
 'P_1STWON_LAST_25_DIFF': 63.17014513367047,
 'P_1STWON_LAST_3_DIFF': 52.53201506591338,
 'P

### Save the final dataset in a CSV

In [ ]:
final_dataset.to_csv("../data/finalDataset.csv", index=False)
